In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

In [ ]:
img_size = (224, 224) 
batch_size = 32
path = './data/weather class/dataset'

# --- Dataset ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)


Found 6862 files belonging to 11 classes.
Using 5490 files for training.
Found 6862 files belonging to 11 classes.
Using 1372 files for validation.
Classes: ['dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'sandstorm', 'snow']
9406464/9406464 [==============================] - 8s 1us/step
Epoch 1/30
172/172 [==============================] - 65s 299ms/step - loss: 1.0110 - accuracy: 0.6559 - val_loss: 0.6674 - val_accuracy: 0.7544
Epoch 2/30
172/172 [==============================] - 49s 288ms/step - loss: 0.6276 - accuracy: 0.7729 - val_loss: 0.6240 - val_accuracy: 0.7631
Epoch 3/30
172/172 [==============================] - 49s 288ms/step - loss: 0.5676 - accuracy: 0.7958 - val_loss: 0.5944 - val_accuracy: 0.7755
Epoch 4/30
172/172 [==============================] - 49s 286ms/step - loss: 0.5110 - accuracy: 0.8124 - val_loss: 0.5912 - val_accuracy: 0.7711
Epoch 5/30
172/172 [==============================] - 49s 286ms/step - loss: 0.4815 - accuracy

In [ ]:
# Normalize + prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (tf.keras.layers.Rescaling(1./255)(x), y))
val_ds = val_ds.map(lambda x, y: (tf.keras.layers.Rescaling(1./255)(x), y))
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# --- Transfer Learning Model ---
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False   # freeze backbone

model = models.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# --- Training ---
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[early_stopping]
)

# --- Evaluation ---
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {acc:.2f}, Loss: {loss:.2f}")

In [25]:

import warnings
warnings.filterwarnings('ignore')
# Unfreeze base model
base_model.trainable = True

# Re-compile with lower learning rate (important!)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # smaller LR
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Train again (fine-tune)
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
172/172 [==============================] - 72s 362ms/step - loss: 0.9026 - accuracy: 0.7000 - val_loss: 0.5423 - val_accuracy: 0.8214
Epoch 2/20
172/172 [==============================] - 60s 348ms/step - loss: 0.5716 - accuracy: 0.7989 - val_loss: 0.5381 - val_accuracy: 0.8214
Epoch 3/20
172/172 [==============================] - 59s 345ms/step - loss: 0.4788 - accuracy: 0.8333 - val_loss: 0.5128 - val_accuracy: 0.8302
Epoch 4/20
172/172 [==============================] - 60s 348ms/step - loss: 0.4330 - accuracy: 0.8490 - val_loss: 0.4856 - val_accuracy: 0.8440
Epoch 5/20
172/172 [==============================] - 59s 346ms/step - loss: 0.3957 - accuracy: 0.8672 - val_loss: 0.4708 - val_accuracy: 0.8433
Epoch 6/20
172/172 [==============================] - 60s 351ms/step - loss: 0.3589 - accuracy: 0.8745 - val_loss: 0.4642 - val_accuracy: 0.8491
Epoch 7/20
172/172 [==============================] - 61s 353ms/step - loss: 0.3181 - accuracy: 0.8863 - val_loss: 0.4552 - val_ac

In [26]:
loss, acc = model.evaluate(val_ds)
print(f"Final Accuracy: {acc:.2f}")

# Save model
model.save("weather_classifier.h5")


43/43 [==============================] - 2s 48ms/step - loss: 0.4004 - accuracy: 0.8630
Final Accuracy: 0.86


In [31]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "sample3.jpg"  # your test image
img = tf.keras.utils.load_img(img_path, target_size=(224,224))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) / 255.0  # scale

predictions = model.predict(img_array)
pred_class = class_names[np.argmax(predictions)]

print("Predicted class:", pred_class)


1/1 [==============================] - 0s 27ms/step
Predicted class: sandstorm
